In [1]:
"""Note: run this script when you already have the "Complete Stats.csv" dataframe saved. 
This script finds the latest fights by referencing that table and updates both the "Complete Stats.csv" and 
the "Normalized Stats Table.csv" """

import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

In [2]:
# Getting latest date from our current dataset
current_df = pd.read_csv("Complete Stats.csv")
current_df["dates"] = pd.to_datetime(current_df["dates"])

latest_date = current_df["dates"].max()
print(latest_date)

2025-08-09 00:00:00


In [3]:
r = requests.get("http://www.ufcstats.com/statistics/events/completed?page=all")
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_ = 'b-statistics__section' )
pages = s.find_all('a', class_ = 'b-link b-link_style_black')
hrefs_start = [a['href'] for a in pages]
# Pulling in the 500 latest events.
hrefs_start = hrefs_start[0:499]

In [4]:
# Initializing variables
stats_url = []
fighter1=[]
fighter2=[]
weight = []
method =[]
rounds =[]
times = []
dates = []
locations = []
event = []
index_value = 0

# Getting first date to start the loop
url = hrefs_start[index_value]
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')
rows = rows[1:]
for row in rows:
    date = soup.find('li', class_ = 'b-list__box-list-item')
    date = date.get_text(strip=True)[5:]
    date = pd.to_datetime(date)

# Running while loop to find URLS before our latest date to avoid reading in unneccessary data
while date > latest_date:
    time.sleep(1)
    url = hrefs_start[index_value]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    rows = rows[1:]
    for row in rows:
        cols = row.find_all('td')
        data = [col.get_text(strip=True) for col in cols][6:]
        weight.append(data[0])
        method.append(data[1])
        rounds.append(data[2])
        times.append(data[3])
        fighters = row.find_all('a', class_ ="b-link b-link_style_black")
        names = [a.get_text(strip = True) for a in fighters]
        fighter1.append(names[0])
        fighter2.append(names[1])
        stat_link = row.get('data-link')
        
        if stat_link != '' or stat_link is not None:
            stats_url.append(row.get('data-link'))
        else: stats_url.append("No Link")

        location = soup.find_all('li', class_ = 'b-list__box-list-item')
        locations.append(location[1].get_text(strip=True)[9:])
        date = soup.find('li', class_ = 'b-list__box-list-item')
        date = date.get_text(strip=True)[5:]
        date = pd.to_datetime(date)
        dates.append(date)
        event.append(soup.find('h2').get_text(strip=True))
    index_value += 1

In [5]:
new_df = pd.DataFrame({
"stats_url" : stats_url,
"fighter1" : fighter1,
"fighter2" : fighter2,
"weight" : weight,
"method" : method,
"rounds" : rounds,
"times" : times,
"dates" : dates,
"locations" : locations,
"event" : event
})

len(new_df)

63

In [6]:
# Using URL's from new_df to scrape the rest of the data
fight_details_links = new_df['stats_url']

all_data = []

failed_requests = []

connect_timeout = 6
read_timeout = 60
success_count = 0
failure_count = 0


for url in fight_details_links:
    try:
        time.sleep(1.5)
        r = requests.get(url, timeout=(connect_timeout, read_timeout))
        soup = BeautifulSoup(r.content, 'html.parser')
        max_round = soup.find_all("i", class_ = 'b-fight-details__text-item')
        max_round = max_round[2].get_text().split('Rnd')[0].strip()[-1]
        fight_details = soup.find_all('section', class_ = 'b-fight-details__section js-fight-section')
        fight_details = fight_details[1]
        tds = fight_details.find_all('td')
        row1 = []
        event = soup.find('h2').get_text(strip=True)
        for td in tds:
            p_tags = td.find_all('p')
            row1.append(p_tags[0].get_text(strip=True))
        f1 = row1[0]
        f1_kd = row1[1]
        f1_sigstr = row1[2]
        f1_sigstr_pct = row1[3]
        f1_totstr = row1[4]
        f1_td = row1[5]
        f1_td_pct = row1[6]
        f1_subatt = row1[7]
        f1_rev = row1[8]
        f1_ctrl = row1[9]
        row2 = []
        for td in tds:
            p_tags = td.find_all('p')
            row2.append(p_tags[1].get_text(strip=True))
        f2 = row2[0]
        f2_kd = row2[1]
        f2_sigstr = row2[2]
        f2_sigstr_pct = row2[3]
        f2_totstr = row2[4]
        f2_td = row2[5]
        f2_td_pct = row2[6]
        f2_subatt = row2[7]
        f2_rev = row2[8]
        f2_ctrl = row2[9]
        stats_url = url

        all_data.append({
            'fighter1' : f1,
            'fighter2': f2,
            'f1_kd' : f1_kd,
            'f1_sigstr' : f1_sigstr,
            'f1_sigstr_pct' : f1_sigstr_pct,
            'f1_totstr' : f1_totstr,
            'f1_td' : f1_td,
            'f1_td_pct' : f1_td_pct,
            'f1_subatt' : f1_subatt,
            'f1_rev' : f1_rev,
            'f1_ctrl' : f1_ctrl,
            'f2_kd' : f2_kd,
            'f2_sigstr' : f2_sigstr,
            'f2_sigstr_pct' : f2_sigstr_pct,
            'f2_totstr' : f2_totstr,
            'f2_td' : f2_td,
            'f2_td_pct' : f2_td_pct,
            'f2_subatt' : f2_subatt,
            'f2_rev' : f2_rev,
            'f2_ctrl' : f2_ctrl,
            'event' : event,
            'tot_round' : max_round,
            'stats_url' : stats_url
            })
        success_count += 1
        print(f"Succcess count: {success_count}")

    except Exception as e:
        failure_count += 1
        print(f"Failure: {failure_count}")
        failed_requests.append(url)

Succcess count: 1
Succcess count: 2
Succcess count: 3
Succcess count: 4
Succcess count: 5
Succcess count: 6
Succcess count: 7
Succcess count: 8
Succcess count: 9
Succcess count: 10
Succcess count: 11
Succcess count: 12
Succcess count: 13
Succcess count: 14
Succcess count: 15
Succcess count: 16
Succcess count: 17
Succcess count: 18
Succcess count: 19
Succcess count: 20
Succcess count: 21
Succcess count: 22
Succcess count: 23
Succcess count: 24
Succcess count: 25
Succcess count: 26
Succcess count: 27
Succcess count: 28
Succcess count: 29
Succcess count: 30
Succcess count: 31
Succcess count: 32
Succcess count: 33
Succcess count: 34
Succcess count: 35
Succcess count: 36
Succcess count: 37
Succcess count: 38
Succcess count: 39
Succcess count: 40
Succcess count: 41
Succcess count: 42
Succcess count: 43
Succcess count: 44
Succcess count: 45
Succcess count: 46
Succcess count: 47
Succcess count: 48
Succcess count: 49
Succcess count: 50
Succcess count: 51
Succcess count: 52
Succcess count: 53
Su

In [7]:
new_stats_df = pd.DataFrame(all_data)

In [8]:
# new_df has reliable fighter1 (winner) and fighter2 (loser) designations, treating nulls from the join as incorrect designations 
complete_df = new_df.merge(new_stats_df, on = ['stats_url', 'fighter1', 'fighter2', 'event'], how="left")

In [9]:
mismatched_winner_df = complete_df[complete_df['f2_sigstr'].isna()]
right_match = complete_df[~complete_df['f2_sigstr'].isna()]
print("# of rows that have improper f1 and f2 designations: ", len(mismatched_winner_df))

# of rows that have improper f1 and f2 designations:  27


In [10]:
url_search = mismatched_winner_df['stats_url'].tolist()
corrected_df = new_stats_df[new_stats_df['stats_url'].isin(url_search)]

# Checking work - should equal True
len(corrected_df) == len(mismatched_winner_df)

True

In [11]:
corrected_df = corrected_df.rename(columns={
'fighter1' : 'fighter2',
'fighter2': 'fighter1',
'f1_kd' : 'f2_kd',
'f1_sigstr' : 'f2_sigstr',
'f1_sigstr_pct' : 'f2_sigstr_pct',
'f1_totstr' : 'f2_totstr',
'f1_td' : 'f2_td',
'f1_td_pct' : 'f2_td_pct',
'f1_subatt' : 'f2_subatt',
'f1_rev' : 'f2_rev',
'f1_ctrl' : 'f2_ctrl',
'f2_kd' : 'f1_kd',
'f2_sigstr' : 'f1_sigstr',
'f2_sigstr_pct' : 'f1_sigstr_pct',
'f2_totstr' : 'f1_totstr',
'f2_td' : 'f1_td',
'f2_td_pct' : 'f1_td_pct',
'f2_subatt' : 'f1_subatt',
'f2_rev' : 'f1_rev',
'f2_ctrl' : 'f1_ctrl'
})

In [12]:
complete_df = pd.concat([corrected_df, right_match], ignore_index=True)
len(complete_df)

63

In [13]:
complete_df[['f1_sigstr_landed', 'f1_sigstr_attempt']] = complete_df['f1_sigstr'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_sigstr_landed', 'f2_sigstr_attempt']] = complete_df['f2_sigstr'].str.split(' of ', expand=True).astype(int)

complete_df[['f1_totstr_landed', 'f1_totstr_attempt']] = complete_df['f1_totstr'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_totstr_landed', 'f2_totstr_attempt']] = complete_df['f2_totstr'].str.split(' of ', expand=True).astype(int)

complete_df[['f1_td_landed', 'f1_td_attempt']] = complete_df['f1_td'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_td_landed', 'f2_td_attempt']] = complete_df['f2_td'].str.split(' of ', expand=True).astype(int)


complete_df = complete_df.drop(columns=[
'f1_sigstr', 'f2_sigstr', 'f1_totstr', 'f2_totstr', 'f1_td', 'f2_td'
], axis=1)


complete_df['f1_sigstr_pct'] = complete_df.apply(lambda row: row['f1_sigstr_landed']/row['f1_sigstr_attempt'] * 100 if row['f1_sigstr_attempt'] != 0 else 0, axis=1)
complete_df['f2_sigstr_pct'] = complete_df.apply(lambda row: row['f2_sigstr_landed']/row['f2_sigstr_attempt'] * 100 if row['f2_sigstr_attempt'] != 0 else 0, axis=1)

complete_df['f2_totstr_pct'] = complete_df.apply(lambda row: row['f2_totstr_landed']/row['f2_totstr_attempt'] * 100 if row['f2_totstr_attempt'] != 0 else 0, axis=1)
complete_df['f1_totstr_pct'] = complete_df.apply(lambda row: row['f1_totstr_landed']/row['f1_totstr_attempt'] * 100 if row['f1_totstr_attempt'] != 0 else 0, axis=1)

complete_df['f1_td_pct'] = complete_df.apply(lambda row: row['f1_td_landed']/row['f1_td_attempt'] * 100 if row['f1_td_attempt'] != 0 else 0, axis=1)
complete_df['f2_td_pct'] = complete_df.apply(lambda row: row['f2_td_landed']/row['f2_td_attempt'] * 100 if row['f2_td_attempt'] != 0 else 0, axis=1)

In [14]:
# Get MM:SS time format into just seconds to help aggregate / total time in the fight
def standard_time_format(mmss):
    if mmss == '--':
        return 0
    else: minutes, seconds = map(int, mmss.split(":"))
    return (minutes*60) + seconds

def total_fight_time(row):
    remainder = standard_time_format(row['times'])
    x = row['rounds'] - 1
    return (x*5*60) + remainder

In [15]:
complete_df = complete_df[~complete_df['rounds'].isna()]
complete_df['rounds'] = complete_df['rounds'].astype(int)
complete_df['rounds'].dtypes

dtype('int64')

In [16]:
complete_df['f1_ctrl_sec'] = complete_df['f1_ctrl'].apply(standard_time_format)
complete_df['f2_ctrl_sec'] = complete_df['f2_ctrl'].apply(standard_time_format)


complete_df['tot_fight_secs'] = complete_df.apply(total_fight_time, axis=1)

In [17]:
complete_df['more_totstr_landed'] = complete_df.apply(lambda row: 'fighter1' if row['f1_totstr_landed'] > row['f2_totstr_landed'] else ('fighter2' if row['f1_totstr_landed'] < row['f2_totstr_landed'] else "equal"), axis=1)
complete_df['more_totstr_attempt'] = complete_df.apply(lambda row: 'fighter1' if row['f1_totstr_attempt'] > row['f2_totstr_attempt'] else ('fighter2' if row['f1_totstr_attempt'] < row['f2_totstr_attempt'] else "equal"), axis=1)

complete_df['more_sigstr_attempt'] = complete_df.apply(lambda row: 'fighter1' if row['f1_sigstr_attempt'] > row['f2_sigstr_attempt'] else ('fighter2' if row['f1_sigstr_attempt'] < row['f2_sigstr_attempt'] else "equal"),axis=1)
complete_df['more_sigstr_landed'] = complete_df.apply(lambda row: 'fighter1' if row['f1_sigstr_landed'] > row['f2_sigstr_landed'] else ('fighter2' if row['f1_sigstr_landed'] < row['f2_sigstr_landed'] else "equal"),axis=1)

In [18]:
def consolidate_methods(x):
    if "SUB" in x:
        return "SUB"
    elif "KO" in x:
        return "KO/TKO"
    elif "DEC" in x:
        return "DEC"
    else: 
        return "DQ/CNC/Overturned/Other"

In [19]:
new_total_stats = pd.read_csv("Complete Stats.csv")

In [20]:
# Add to Complete Stats Table
new_total_stats = pd.concat([current_df, complete_df], ignore_index=True)

new_total_stats['method'] = new_total_stats['method'].apply(consolidate_methods)

new_total_stats.drop_duplicates(inplace=True)

new_total_stats.to_csv("Complete Stats.csv", index=False)

In [206]:
# current_df['rounds'] = current_df['rounds'].astype(int)
# current_df['tot_fight_secs'] = current_df.apply(total_fight_time, axis=1)
# current_df.to_csv("Complete Stats.csv")

In [21]:
new_total_stats['dates'] = pd.to_datetime(new_total_stats['dates'])


#new_total_stats.sort_values(by='dates', ascending=False).head(23)

In [22]:
stats_df = pd.read_csv("Complete Stats.csv")

# separate into two tables then append each other to normalize the data
# stats_df.columns
f1_df = stats_df[['event', 'fighter1', 'weight', 'rounds',
       'times', 'method', 'locations', 'dates', 'stats_url', 'f1_kd',
       'f1_sigstr_pct', 'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl',
       'f1_sigstr_landed', 'f1_sigstr_attempt', 'f1_totstr_landed', 'f1_totstr_attempt', 'f1_td_landed',
       'f1_td_attempt',
       'f1_totstr_pct', 'f1_ctrl_sec', 'more_totstr_landed',
       'more_totstr_attempt', 'more_sigstr_attempt', 'more_sigstr_landed', 'tot_fight_secs', 'tot_round']].copy()

f1_df["is_winner"] = True
f1_df["fighter_num"] = "fighter1"
f1_df.rename(columns={"fighter1" : "fighter" , 'f1_kd' : 'kd',
       'f1_sigstr_pct' : 'sigstr_pct', 'f1_td_pct' : 'td_pct', 'f1_subatt' : 'subatt', 'f1_rev' : 'rev', 'f1_ctrl' : 'ctrl', 'f1_sigstr_landed' : 'sigstr_landed',
       'f1_sigstr_attempt' : 'sigstr_attempt',
       'f1_totstr_landed' : 'totstr_landed', 'f1_totstr_attempt' : 'totstr_attempt', 'f1_td_landed' : 'td_landed', 'f1_td_attempt' : 'td_attempt', 'f1_totstr_pct' : 'totstr_pct', 'f1_ctrl_sec' : 'ctrl_sec'}, inplace=True)

f2_df = stats_df[['event', 'fighter2', 'weight', 'rounds',
       'times', 'method', 'locations', 'dates', 'stats_url', 'f2_kd',
       'f2_sigstr_pct', 'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'f2_sigstr_landed',
       'f2_sigstr_attempt',
       'f2_totstr_landed', 'f2_totstr_attempt', 'f2_td_landed', 'f2_td_attempt', 'f2_totstr_pct', 'f2_ctrl_sec', 'more_totstr_landed',
       'more_totstr_attempt', 'more_sigstr_attempt', 'more_sigstr_landed','tot_fight_secs','tot_round']].copy()

f2_df["is_winner"] = False
f2_df["fighter_num"] = "fighter2"
f2_df.rename(columns={"fighter2" : "fighter", "f2_kd": "kd",
       'f2_sigstr_pct': 'sigstr_pct', 'f2_td_pct': 'td_pct', 'f2_subatt' : 'subatt', 'f2_rev': 'rev', 'f2_ctrl': 'ctrl', 'f2_sigstr_landed': 'sigstr_landed',
       'f2_sigstr_attempt' : 'sigstr_attempt',
       'f2_totstr_landed' : 'totstr_landed', 'f2_totstr_attempt' : 'totstr_attempt', 'f2_td_landed' : 'td_landed', 'f2_td_attempt' : 'td_attempt', 'f2_totstr_pct' : 'totstr_pct', 'f2_ctrl_sec' : 'ctrl_sec'}, inplace=True)

In [23]:
norm_df = pd.concat([f1_df, f2_df], ignore_index = True)
norm_df['method'] = norm_df['method'].apply(consolidate_methods)
norm_df.to_csv("Normalized Stats Table.csv", index=False)